In [1]:
import os
import pandas as pd
from pathlib import Path    

In [66]:
# Define the directory path for the CSV files
directory_path = "../Zoom_Downloads"

# Initialize a dictionary to keep track of schema matches and a counter for rows
schemas = {}
total_rows_before_merge = 0

# List to keep track of dataframes for merging
dataframes_to_merge = []

In [67]:
# Function to get the schema of a CSV file
def get_schema(file_path):
    try:
        df = pd.read_csv(file_path, nrows=1, low_memory=False)  # read only the first row to get the schema
        return df.columns.tolist()  # return the column headers as a list
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Function to count the rows of a CSV file
def count_rows(file_path):
    try:
        return pd.read_csv(file_path, low_memory=False).shape[0]  # return the number of rows
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return 0

# Process the files
for file_name in os.listdir(directory_path):
    if file_name.endswith('.csv'):  # Check only csv files
        file_path = Path(directory_path) / file_name
        schema = get_schema(file_path)
        if schema is not None:
            schema_key = tuple(schema)  # convert the list to a tuple to use as a dict key
            if schema_key in schemas:
                # If schema matches, append the dataframe to the list and update row count
                dataframes_to_merge.append(pd.read_csv(file_path, low_memory=False))
                total_rows_before_merge += count_rows(file_path)
            else:
                # If new schema, print the file name and add to the dict
                print(file_name)
                schemas[schema_key] = file_name

# Merge all dataframes with the same schema
merged_df = pd.concat(dataframes_to_merge, ignore_index=True) if dataframes_to_merge else None

# Function to compare row count and remove duplicates
def process_merged_dataframe(merged_df, total_rows_before_merge):
    if merged_df is not None:
        # Check for discrepancies in row count
        total_rows_after_merge = merged_df.shape[0]
        if total_rows_before_merge != total_rows_after_merge:
            print(f"Row count mismatch: {total_rows_before_merge} before, {total_rows_after_merge} after merge.")
        
        # Remove duplicates
        merged_df.drop_duplicates(inplace=True)
        print(f"Removed duplicates. Rows after deduplication: {merged_df.shape[0]}.")

# Check if we have a merged dataframe, process it, and output a file if we do
output_file_path = None
if merged_df is not None:
    process_merged_dataframe(merged_df, total_rows_before_merge)
    output_file_path = Path(directory_path) / 'merged_files.csv'
    merged_df.to_csv(output_file_path, index=False)

# The output file path will point to the final merged CSV
output_file_path

Target Companies NYC BOS PHL (2).csv
Banking EU-7UywK6-enhance.csv
Removed duplicates. Rows after deduplication: 187650.


PosixPath('../Zoom_Downloads/merged_files.csv')

In [68]:
#reference file
file_path_df1 = "../Zoom_Downloads/249_100423_PERSON (1).csv"

#To Change 
file_path_df2 = "../Zoom_Downloads/Target Companies NYC BOS PHL (2).csv"


In [69]:
# Load the first file and get the info
df1 = pd.read_csv(file_path_df1)
df1_info = df1.columns.tolist()

# Load the second file and get the info
df2 = pd.read_csv(file_path_df2, low_memory=False)
df2_info = df2.columns.tolist()

# Compare and highlight differences
unique_to_df1 = set(df1_info) - set(df2_info)
unique_to_df2 = set(df2_info) - set(df1_info)
common_columns = set(df1_info) & set(df2_info)

# Count the unique and common columns
unique_to_df1_count = len(unique_to_df1)
unique_to_df2_count = len(unique_to_df2)
common_columns_count = len(common_columns)

print(f"Columns unique to DF1 (reference) ({unique_to_df1_count}):")
print(unique_to_df1)
print(f"\nColumns unique to DF2 (to change) ({unique_to_df2_count}):")
print(unique_to_df2)
print(f"\nCommon columns ({common_columns_count}):")
print(common_columns)


Columns unique to DF1 (reference) (0):
set()

Columns unique to DF2 (to change) (0):
set()

Common columns (71):
{'Facebook Company Profile URL', 'Person State', 'Company Country', 'Revenue (in 000s USD)', 'LinkedIn Contact Profile URL', 'Twitter Company Profile URL', 'Full Address', 'Last Name', 'Ownership Type', 'Direct Phone Number', 'SIC Codes', 'Secondary Industry Hierarchical Category', 'Founded Year', 'Number of Locations', 'Recent Funding Round', 'Email Domain', 'Contact Accuracy Grade', 'Middle Name', 'Recent Funding Date', 'Industry Hierarchical Category', 'ZoomInfo Contact ID', 'Fax', 'All Sub-Industries', 'Alexa Rank', 'ZoomInfo Contact Profile URL', 'Revenue Range (in USD)', 'All Industries', 'Notice Provided Date', 'Primary Sub-Industry', 'Company HQ Phone', 'Suffix', 'SIC Code 2', 'LinkedIn Company Profile URL', 'Company Street Address', 'Company State', 'Management Level', 'NAICS Code 2', 'Certified Active Company', 'Certification Date', 'Recent Investors', 'Company Cit

In [ ]:
columns_to_drop = ['']

df2 = pd.read_csv(file_path_df2, low_memory=False)
df2 = df2.drop(columns_to_drop, axis=1)
df2.to_csv(file_path_df2, index=False)


In [33]:
columns_to_add = {'Query Name': 'none'} 
df2 = pd.read_csv(file_path_df2, low_memory=False)

# Add the new column(s)
for column_name, default_value in columns_to_add.items():
    df2[column_name] = default_value

df2.to_csv(file_path_df2, index=False)


In [ ]:
### Re-Ordering of the columns

In [60]:
# Load the first file and get its column order
df1 = pd.read_csv(file_path_df1)
df1_info = df1.columns.tolist()

# Load the second file
df2 = pd.read_csv(file_path_df2, low_memory=False)

# Reorder df2's columns to match df1's order, dropping any that don't exist in df1 and adding missing ones as NaN
df2_reordered = df2.reindex(columns=df1_info).assign(**{col: pd.NA for col in df1_info if col not in df2.columns})

# Save the reordered df2 back to CSV
df2_reordered.to_csv(file_path_df2, index=False)


In [61]:
### Matching type with DF1

In [62]:
# Load the first file
df1 = pd.read_csv(file_path_df1)

# Determine the data types in df1
dtype_dict = df1.dtypes.to_dict()

# Load the second file
df2 = pd.read_csv(file_path_df2, low_memory=False)

# Replace NaN values with an appropriate value based on the column data type
for col in df2.columns:
    if col in dtype_dict:
        if dtype_dict[col] == 'float64' or dtype_dict[col] == 'int64':
            replacement_value = 0  # Or df2[col].mean() for mean replacement
        elif dtype_dict[col] == 'object':
            replacement_value = 'Unknown'
        df2[col].fillna(replacement_value, inplace=True)

# Now attempt to convert data types of df2 columns to match df1 where possible
for col in df2.columns:
    if col in dtype_dict:
        try:
            df2[col] = df2[col].astype(dtype_dict[col])
        except ValueError as e:
            print(f"Could not convert column {col} to {dtype_dict[col]} due to: {e}")

# Save the NaN-replaced and converted df2 back to CSV
df2.to_csv(file_path_df2, index=False)


In [63]:
import pandas as pd

# Assuming df2 is already loaded
# file_path_df2 = "../Zoom_Downloads/Gothenburg list-vR47vY-enhance.csv"
# df2 = pd.read_csv(file_path_df2, low_memory=False)

# Count NaN values for each column
nan_count_per_column = df2.isna().sum()

# Count total NaN values in the DataFrame
total_nan_count = df2.isna().sum().sum()

print("NaN count per column:\n", nan_count_per_column)
print("\nTotal NaN count in df2:", total_nan_count)


NaN count per column:
 ZoomInfo Contact ID    0
Last Name              0
First Name             0
Middle Name            0
Salutation             0
                      ..
Company Zip Code       0
Company Country        0
Full Address           0
Number of Locations    0
Query Name             0
Length: 71, dtype: int64

Total NaN count in df2: 0


In [64]:

nan_count_per_column = df2.isna().sum()
total_rows = len(df2)

# Calculate the percentage of NaN values per column
nan_percentage_per_column = (nan_count_per_column / total_rows) * 100

# Display the percentage of NaN values per column
print("Percentage of NaN values per column:\n", nan_percentage_per_column)


Percentage of NaN values per column:
 ZoomInfo Contact ID    0.0
Last Name              0.0
First Name             0.0
Middle Name            0.0
Salutation             0.0
                      ... 
Company Zip Code       0.0
Company Country        0.0
Full Address           0.0
Number of Locations    0.0
Query Name             0.0
Length: 71, dtype: float64
